In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from scipy import stats

In [ ]:
import datetime
import json
import urllib
import pandas as pd
import time
import numpy as np
import warnings


token = 'LRC3I7BREUJV9NU8'
warnings.filterwarnings("ignore")

def request_until_succeed(url):
    success = False
    while success is False:
        response = urllib.request.urlopen(url)
        if response.getcode() == 200:
            success = True
    return response.read()


def data_collect(symbol_list, since, until, value):

    opts = {
        'open': '1. open',
        'high': '2. high',
        'low': '3. low',
        'close': '4. close',
        'adj_close': '5. adjusted close'}

    body = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='
    tail = '&outputsize=full&apikey='

    dates = []
    start = datetime.datetime.strptime(since, '%Y-%m-%d')
    end = datetime.datetime.strptime(until, '%Y-%m-%d')
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days + 1)]

    for date in date_generated:
        dates.append(date.strftime("%Y-%m-%d"))

    dataset = pd.DataFrame()
    dataset['date'] = dates
    dataset = dataset.set_index(dataset['date'])
    dataset = dataset.drop('date', axis=1)

    count = 1
    for symbol in symbol_list:
        print (str(count) + '. ' + symbol)
        url = body + symbol + tail + token
        data = json.loads(request_until_succeed(url))
        dataset[symbol] = 0.0000
        for trade in dataset.index:
            if 'Time Series (Daily)' in data and trade in data['Time Series (Daily)']:
                dataset.loc[trade, symbol] = data['Time Series (Daily)'][trade][opts[value]]
        count += 1
    dataset = dataset[(dataset.T != 0).any()]

    return dataset



In [ ]:
import re
count = 0

tickers = pd.read_csv('TSX_Constituents_05202019.csv')
tickers = list(tickers.Symbol)
symbol_list = []
for ticker in tickers:
    test2 = str(ticker).endswith('.UN')
    test3 = str(ticker).endswith('.X')
    test4 = str(ticker).endswith('.B')
    test5 = str(ticker).endswith('.A')
    if not test2 and not test3 and not test4 and not test5:
        ticker = str(ticker) + '.TO'
    symbol_list.append(ticker)

In [ ]:
since = '2019-05-15'
until = '2019-05-16'
value = 'adj_close'
dataset = data_collect(symbol_list, since, until, value)

In [ ]:
dataset

In [ ]:
since = '2019-05-15'
until = '2019-05-16'
value = 'adj_close'
dataset = data_collect(['ASR.TO'], since, until, value)
dataset